In [1]:
import numpy as np

In [10]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoidDerivative(x):
    return x * (1 - x)

class Neuron():

    def __init__(self, sizeIn, sizeOut, activationFunction=sigmoid, errorFunction=sigmoidDerivative, learningRate=0.1):
        self.learningRate = learningRate
        self.sizeIn = sizeIn
        self.sizeOut = sizeOut
        self.errorFunction = errorFunction
        self.activationFunction = activationFunction
        self.W = np.random.uniform(size=(sizeIn, sizeOut))
        self.b = np.random.uniform(size=(1, sizeOut))
    
    def forward(self, x):
        self.inputVal = x
        self.outputVal = self.activationFunction(x.dot(self.W) + self.b)
        return self.outputVal
    
    def delta(self, error):
        self._loss = error * self.errorFunction(self.outputVal)
        return self._loss
    
    def backward(self, x, delta):
        self.W += x.T.dot(delta) * self.learningRate
        self.b += np.sum(delta, axis=0, keepdims=True) * self.learningRate

class Model():

    def __init__(self, inputSize, hiddenSize, outputSize, activationFunction=sigmoid, errorFunction=sigmoidDerivative, learningRate=0.1):
        self.learningRate = learningRate
        self.inputSize = inputSize
        self.hiddenSize = hiddenSize
        self.outputSize = outputSize
        self.activationFunction = activationFunction
        self.errorFunction = errorFunction

        self.hiddenLayer = Neuron(inputSize, hiddenSize, self.activationFunction, self.errorFunction, self.learningRate)
        self.outputLayer = Neuron(hiddenSize, outputSize, self.activationFunction, self.errorFunction, self.learningRate)
    
    def forward(self, x):
        self.hiddenOutputVal = self.hiddenLayer.forward(x)
        self.outputVal = self.outputLayer.forward(self.hiddenOutputVal)
        return self.outputVal

    def backward(self, x, err):
        delta_outputLayer = self.outputLayer.delta(err)
        error_hiddenLayer = delta_outputLayer.dot(self.outputLayer.W.T)
        delta_hiddenLayer = self.hiddenLayer.delta(error_hiddenLayer)

        self.hiddenLayer.backward(x, delta_hiddenLayer)
        self.outputLayer.backward(self.hiddenOutputVal, delta_outputLayer)
    
    def result(self):
        return self.outputVal

    @staticmethod
    def loss(predRes, trueRes):
        return trueRes - predRes
        


In [11]:
import tqdm

epochs = 10000

np.random.seed(0)
md = Model(2, 2, 1)

inputs = np.array([[0,0],[0,1],[1,0],[1,1]])
expected_output = np.array([[0],[1],[1],[0]])

for _ in tqdm.tqdm(range(epochs)):
    y = md.forward(inputs)
    err = Model.loss(y, expected_output)
    md.backward(inputs, err)

md.result()

100%|██████████| 10000/10000 [00:00<00:00, 19193.89it/s]


array([[0.05770383],
       [0.9470198 ],
       [0.9469948 ],
       [0.05712647]])

----------------

In [85]:
class Layer():

    def __init__(self, learningRate, sizeIn, sizeOut, activationFunction, errorFunction):
        self.learningRate = learningRate
        self.sizeIn = sizeIn
        self.sizeOut = sizeOut
        self.errorFunction = errorFunction
        self.activationFunction = activationFunction
        self.W = np.random.uniform(size=(sizeIn, sizeOut))
        self.b = np.random.uniform(size=(1, sizeOut))
    
    def getOutput(self):
        return self.activationFunction(self.activaValue)

    def forward(self, x):
        self.inVal = x
        self.activaValue = np.dot(x, self.W) + self.b
        return self.getOutput()

    def backward(self, output_delta, output_weights):
        error = output_delta.dot(output_weights.T)
        deltaLayer = error * self.errorFunction(self.getOutput())
        self.W += self.inVal.T.dot(deltaLayer) * self.learningRate
        self.b += np.sum(deltaLayer, axis=0, keepdims=True) * self.learningRate

class OutputLayer():
    
    def __init__(self, learningRate, sizeIn, sizeOut, activationFunction, errorFunction):
        self.learningRate = learningRate
        self.sizeIn = sizeIn
        self.sizeOut = sizeOut
        self.errorFunction = errorFunction
        self.activationFunction = activationFunction
        self.W = np.random.uniform(size=(sizeIn, sizeOut))
        self.b = np.random.uniform(size=(1, sizeOut))
    
    def getOutput(self):
        return self.activationFunction(self.activaValue)

    def forward(self, x):
        self.inVal = x
        self.activaValue = np.dot(x, self.W) + self.b
        return self.getOutput()
    
    def backward(self, y):
        error = y - self.getOutput()
        delta = error * self.errorFunction(self.getOutput())
        prevWeigths = self.W.copy()
        self.W += self.inVal.T.dot(delta) * self.learningRate
        self.b += np.sum(delta, axis=0, keepdims=True) * self.learningRate
        return delta, prevWeigths


In [86]:
import tqdm

np.random.seed(0)

inputs = np.array([[0,0],[0,1],[1,0],[1,1]])
expected_output = np.array([[0],[1],[1],[0]])

epochs = 10000

hidden_layer = Layer(learningRate=0.1, sizeIn=2, sizeOut=2, activationFunction=sigmoid, errorFunction=sigmoidDerivative)
ouput_layer = OutputLayer(learningRate=0.1, sizeIn=2, sizeOut=1, activationFunction=sigmoid, errorFunction=sigmoidDerivative)


for _ in tqdm.tqdm(range(epochs)):
    hidden_layer_output = hidden_layer.forward(inputs)
    output_layer_output = ouput_layer.forward(hidden_layer_output)

    delta_out, weights_out = ouput_layer.backward(expected_output)
    hidden_layer.backward(delta_out, weights_out)

print(ouput_layer.getOutput())

100%|██████████| 10000/10000 [00:00<00:00, 15431.83it/s]

[[0.05770383]
 [0.9470198 ]
 [0.9469948 ]
 [0.05712647]]
